In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import glob, os
import numpy as np
import matplotlib.pyplot as plt
import datetime  as dt
import seaborn as sns

## Data Directory

In [7]:
sports = 'NCAA'

In [10]:
data_directory = os.path.join('..','data','sports_data/{}/uh08_17.csv'.format(sports))
data_directory_saves = os.path.join( '..','data','clean_data','sports_data/{}/'.format(sports))

In [11]:
df = pd.read_csv(data_directory)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 16 columns):
G             130 non-null int64
Date          130 non-null object
Time          65 non-null object
Day           130 non-null object
School        130 non-null object
Unnamed: 5    69 non-null object
Opponent      130 non-null object
Conf          130 non-null object
Unnamed: 8    130 non-null object
Pts           130 non-null int64
Opp           130 non-null int64
W             130 non-null int64
L             130 non-null int64
Streak        130 non-null object
TV            31 non-null object
Notes         6 non-null object
dtypes: int64(5), object(11)
memory usage: 16.3+ KB


In [13]:
df.head()

,G,Date,Time,Day,School,Unnamed: 5,Opponent,Conf,Unnamed: 8,Pts,Opp,W,L,Streak,TV,Notes
0,1,Sep 9 2017,10:30 PM,Sat,Houston,@,Arizona,Pac-12,W,19,16,1,0,W 1,NaN,NaN
1,2,Sep 16 2017,8:00 PM,Sat,Houston,NaN,Rice,CUSA,W,38,3,2,0,W 2,NaN,NaN
2,3,Sep 23 2017,12:00 PM,Sat,Houston,NaN,Texas Tech,Big 12,L,24,27,2,1,L 1,NaN,NaN
3,4,Sep 30 2017,12:00 PM,Sat,Houston,@,Temple,American,W,20,13,3,1,W 1,NaN,NaN
4,5,Oct 7 2017,7:00 PM,Sat,Houston,NaN,SMU,American,W,35,22,4,1,W 2,NaN,NaN


## Drop columns

In [16]:
list(df.columns)

['G',
 'Date',
 'Time',
 'Day',
 'School',
 'Unnamed: 5',
 'Opponent',
 'Conf',
 'Unnamed: 8',
 'Pts',
 'Opp',
 'W',
 'L',
 'Streak',
 'TV',
 'Notes']

In [14]:
df.drop(['G','Time',
 'Day',
 'Conf',
 'W',
 'L',
 'Streak',
 'Notes',
 'Unnamed: 15'],axis=1,inplace=True)

KeyError: "['Unnamed: 15'] not found in axis"

In [15]:
df.head()

,G,Date,Time,Day,School,Unnamed: 5,Opponent,Conf,Unnamed: 8,Pts,Opp,W,L,Streak,TV,Notes
0,1,Sep 9 2017,10:30 PM,Sat,Houston,@,Arizona,Pac-12,W,19,16,1,0,W 1,NaN,NaN
1,2,Sep 16 2017,8:00 PM,Sat,Houston,NaN,Rice,CUSA,W,38,3,2,0,W 2,NaN,NaN
2,3,Sep 23 2017,12:00 PM,Sat,Houston,NaN,Texas Tech,Big 12,L,24,27,2,1,L 1,NaN,NaN
3,4,Sep 30 2017,12:00 PM,Sat,Houston,@,Temple,American,W,20,13,3,1,W 1,NaN,NaN
4,5,Oct 7 2017,7:00 PM,Sat,Houston,NaN,SMU,American,W,35,22,4,1,W 2,NaN,NaN


## change date to datetime index

In [ ]:
# Map the lowering function to all column names
df.columns = map(str.lower, df.columns)

In [ ]:
df.date = pd.to_datetime(df['date'])
df = df.set_index('date').sort_index(ascending=True)

In [ ]:
df.head()

## chane unnamed:5 to home_away

In [ ]:
df.rename(columns={
    'unnamed: 5': 'home_away',
    'unnamed: 8': 'win_lost',
}, inplace=True)

In [ ]:
df.home_away.fillna('home',inplace=True)

In [ ]:
df.head()

## select home games

In [ ]:
home_games = df.home_away == 'home'
df = df[home_games]
df.drop(['home_away'],axis=1,inplace=True)

In [ ]:
df.head()

## clean columns

In [ ]:
df.school.unique()

In [ ]:
df.opponent.unique()

## remove extra characters line

In [ ]:
df.opponent = df.opponent.str.replace(r"\(.*\)"," ")

In [ ]:
df.opponent.unique()

In [ ]:
df.win_lost.unique()

In [ ]:
df.pts.unique()

In [ ]:
df.opp.unique()

## display null values

In [ ]:
df.apply(lambda x: sum(x.isnull()))

## save dataset

In [ ]:
df.to_csv(data_directory_saves+'Rice_2010_2017.csv')